In [1]:
import wandb
from datetime import datetime

In [2]:
mask_model_ver = 132508
lr = 5e-4
batch_size = 32
wd = 1e-5
IMAGE_SIZE = 512
backbone='efficientdetv2_ds'
vertical_type = "axial"
train_portion = 0.5
milestones = [10,20,30,40]
model_name = "effdet"
epochs =50



In [3]:
wandb_entity='longyi'
model_name = "effdet"
wandb.init(project="YOLOR", entity=wandb_entity, config={
    "model":model_name,
    "batch_size":batch_size,
    "lr" : lr,
    "wd" : wd,
    "backbone" : backbone,
    "image_size" : IMAGE_SIZE,
})
wandb.run.name = f'axial_detection_{model_name}_' + datetime.now().strftime("%H%M%S")
wandb.run.name

wandb: Currently logged in as: longyi. Use `wandb login --relogin` to force relogin


'axial_detection_effdet_193536'

In [4]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import sys

effdet_path = "../third/effdet"
sys.path.append(effdet_path)
timm_path = "../third/timm-pytorch-image-models"
sys.path.append(timm_path)
import timm
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from matplotlib import patches
import sklearn
import os
from tqdm import tqdm

omega_path = "../third/omegaconf"
sys.path.append(omega_path)
from omegaconf import OmegaConf
import glob
import sklearn
import math
import random

from PIL import Image

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
import torchvision.transforms as T
import torchvision.transforms.functional as TF

from effdet import create_model
from effdet import DetBenchPredict
# from transformers import get_cosine_schedule_with_warmup

import warnings

warnings.filterwarnings('ignore')
from sklearn import metrics, model_selection, preprocessing

import json
# from sklearn.model_selection import GroupKFold

%load_ext autoreload
%autoreload 2

# pos_weight = torch.tensor(pos_weight)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
DATA_DIR = "/root/autodl-tmp/cervical_spine/"
IMAGES_DIR = os.path.join(DATA_DIR, 'train_axial_images_jpeg95_croped_132508')
MASK_DIR = os.path.join(DATA_DIR, f'segmentation_axial_results_{mask_model_ver}')


checkpoint = os.path.join(DATA_DIR, 'checkpoint', 'axial_detection_effdet_104709-epoch-54.pth')

In [10]:
def get_det_model(image_size, checkpoint=None):
    if checkpoint is None:

        model = create_model(backbone , bench_task='train' , num_classes= 1 , image_size=(image_size, image_size), bench_labeler=True, pretrained=True)
        model = model.to(device)
        return model
    else:
        model = create_model(backbone , bench_task='train' , num_classes= 1 , image_size=(image_size, image_size), bench_labeler=True, pretrained=True)
        state = torch.load(os.path.join(DATA_DIR, 'checkpoint', checkpoint), map_location=torch.device(device))
        model.load_state_dict(state["model"])
        model = model.to(device)
        return model

model = get_det_model(IMAGE_SIZE, checkpoint)

In [11]:
from evaluate import *
evaluator = Evaluate(model.model, save_dir=os.path.join(DATA_DIR, 'predictions', wandb.run.name))

In [13]:
start_batch_index = 0
epoch=0
for i in range(252):
    loss, pos_loss, neg_loss, batch_index = evaluator.evaluate(epoch + i + start_batch_index)
    wandb.log({
        'val/total_loss': loss,
        'val/pos_loss': pos_loss,
        'val/neg_loss': neg_loss,
        'val/batch_index' : batch_index
    })

test df length :  2819


100%|██████████| 177/177 [00:30<00:00,  5.87it/s]


loss :1.2904549226283015 pos_loss:0.5378107215066045 neg_loss:0.8833464724241963
test df length :  2429


100%|██████████| 152/152 [00:24<00:00,  6.27it/s]


loss :0.3303673717719574 pos_loss:0.23327259213386037 neg_loss:0.2561323552892014
test df length :  3430


...

100%|██████████| 167/167 [00:26<00:00,  6.23it/s]


loss :0.8456385525623664 pos_loss:0.4373085920186616 neg_loss:0.6578373105562866
test df length :  1074


100%|██████████| 68/68 [00:14<00:00,  4.75it/s]

loss :0.7653869232139133 pos_loss:0.38496668619848606 neg_loss:0.7734576821084108



wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
